# Lab 07.04 : Test set evaluation -- solution

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import utils

### Download the CIFAR dataset -- check the size carefully!

In [2]:
from utils import check_cifar_dataset_exists
data_path=check_cifar_dataset_exists()

train_data=torch.load(data_path+'cifar/train_data.pt')
train_label=torch.load(data_path+'cifar/train_label.pt')
test_data=torch.load(data_path+'cifar/test_data.pt')
test_label=torch.load(data_path+'cifar/test_label.pt')

print(train_data.size())

torch.Size([50000, 3, 32, 32])


### Make a ONE layer net class. 

In [3]:
class one_layer_net(nn.Module):

    def __init__(self, input_size, output_size):
        super(one_layer_net , self).__init__()
        self.linear_layer = nn.Linear( input_size, output_size , bias=True)
        
    def forward(self, x):
        scores = self.linear_layer(x)
        return scores

### Build the net. 

In [4]:
net=one_layer_net(3072,10)
print(net)

one_layer_net(
  (linear_layer): Linear(in_features=3072, out_features=10, bias=True)
)


### Choose the criterion and optimizer. Also choose:


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD( net.parameters() , lr=0.01 )
bs=20

### Do 50 passes through the training set.

In [6]:
for epoch in range(50):
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(50000)
 
    for count in range(0,50000,bs):
    
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch       
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices]
        minibatch_label= train_label[indices]

        # reshape the minibatch
        inputs = minibatch_data.view(bs,3072)

        # tell Pytorch to start tracking all operations that will be done on "inputs"
        inputs.requires_grad_()

        # forward the minibatch through the net 
        scores=net( inputs ) 

        # Compute the average of the losses of the data points in the minibatch
        loss =  criterion( scores , minibatch_label) 
        
        # backward pass to compute dL/dU, dL/dV and dL/dW   
        loss.backward()

        # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
        optimizer.step()
        
        # START COMPUTING STATS
        
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item()
        
        # compute the error made on this batch and add it to the running error       
        error = utils.get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
    
    
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    
    print('epoch=',epoch, '\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
        
    
        
        

epoch= 0 	 loss= 1.9606445921421052 	 error= 69.72800000667571 percent
epoch= 1 	 loss= 1.86200466299057 	 error= 65.27199998378754 percent
epoch= 2 	 loss= 1.8408433421611785 	 error= 64.29999990940094 percent
epoch= 3 	 loss= 1.8168387707710265 	 error= 63.289999988079074 percent
epoch= 4 	 loss= 1.8066944542884826 	 error= 62.80400000095368 percent
epoch= 5 	 loss= 1.7945659608364106 	 error= 62.18799994707107 percent
epoch= 6 	 loss= 1.788796676683426 	 error= 61.978000006675714 percent
epoch= 7 	 loss= 1.7866448153018952 	 error= 61.93799997329712 percent
epoch= 8 	 loss= 1.778327439212799 	 error= 61.56600001335144 percent
epoch= 9 	 loss= 1.7725445796012878 	 error= 61.37400002479553 percent
epoch= 10 	 loss= 1.7689770634651185 	 error= 61.0940000128746 percent
epoch= 11 	 loss= 1.7637074474573136 	 error= 60.77200000524521 percent
epoch= 12 	 loss= 1.7649710573673247 	 error= 60.968000025749205 percent
epoch= 13 	 loss= 1.7562139580011369 	 error= 60.63199997663497 percent
epoc

# You only have to complete this cell

### So we are doing around 59% error on the training set, how much are we doing on test set?

In [7]:
running_error=0
num_batches=0


for i in range(0,10000,bs):

    # extract the minibatch
    minibatch_data =  test_data[i:i+bs]
    minibatch_label= test_label[i:i+bs]

    # reshape the minibatch
    inputs = minibatch_data.view(bs,3072)

    # feed it to the network
    scores=net( inputs ) 

    # compute the error made on this batch
    error = utils.get_error( scores , minibatch_label)
    
    # add it to the running error
    running_error += error.item()

    num_batches+=1


# compute error rate on the full test set
total_error = running_error/num_batches

print( 'error rate on test set =', total_error*100 ,'percent')

error rate on test set = 64.39000008106231 percent


### Choose image at random from the test set and see how good/bad are the predictions

In [8]:
# choose a picture at random
idx=randint(0, 10000-1)
im=test_data[idx]

# diplay the picture
utils.show(im)

# feed it to the net and display the confidence scores
scores =  net( im.view(1,3072)) 
probs= F.softmax(scores, dim=1)
utils.show_prob_cifar(probs)